## Bicicle algorithm

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import pandas as pd
import numpy as np
import reservoirpy as rpy
from typing import Dict, Tuple
import pickle

from src.features.denoise_signal import (
    resample_time_series,
    moving_std_filter,
    holt_winters_filter,
    denoise_signal,
)
from src.features.slice_time_series import create_training_data
from src.models.forecaster import Forecaster
from src import paths

In [2]:
rpy.verbosity(0)

0

In [3]:
signal_df = pd.read_csv(paths.data_processed_dir("test_signal_filtered_dataset_ndvi.csv"), index_col=["ID", "IDpix"])
signal_df.columns = pd.to_datetime(signal_df.columns)
metadata_df = pd.read_csv(paths.data_processed_dir("metadata_filtered_dataset_ndvi.csv"), index_col=["ID", "IDpix"])

In [4]:
signal_df

,,2014-01-01,2014-01-02,2014-01-08,2014-01-09,2014-01-10,2014-01-16,2014-01-17,2014-01-18,2014-01-24,2014-01-25,...,2022-12-08,2022-12-09,2022-12-10,2022-12-14,2022-12-16,2022-12-17,2022-12-18,2022-12-19,2022-12-24,2022-12-25
ID,IDpix,,,,,,,,,,,,,,,,,,,,,
12,377,NaN,NaN,NaN,0.590611,NaN,NaN,0.558519,NaN,NaN,0.740373,...,NaN,0.699372,NaN,0.659060,NaN,0.644077,NaN,NaN,NaN,0.705126
27,745,0.689007,NaN,NaN,0.737172,NaN,NaN,0.674854,NaN,NaN,0.771128,...,NaN,0.699097,NaN,0.628368,NaN,0.668308,NaN,NaN,NaN,0.664748
30,906,0.790094,NaN,NaN,0.791995,NaN,NaN,0.727667,NaN,NaN,0.814491,...,NaN,0.811179,NaN,0.780642,NaN,0.784341,NaN,NaN,NaN,0.758479
46,1199,NaN,NaN,NaN,0.581447,NaN,NaN,0.512926,NaN,0.533878,0.571442,...,0.561523,0.568062,NaN,NaN,0.560768,NaN,NaN,NaN,0.549780,0.541098
28,778,0.706038,NaN,NaN,0.689303,NaN,NaN,0.609940,NaN,NaN,0.765705,...,NaN,0.642517,NaN,0.683086,NaN,0.711361,NaN,NaN,NaN,0.767881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1732,0.553579,NaN,NaN,0.540444,NaN,NaN,NaN,NaN,NaN,0.585871,...,NaN,0.525082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.510168
371,687,0.691285,NaN,NaN,0.754736,NaN,NaN,0.646313,NaN,NaN,0.745945,...,NaN,0.699709,NaN,0.673036,NaN,0.690027,NaN,NaN,NaN,0.670598
393,4029,0.671869,NaN,NaN,0.659421,NaN,NaN,NaN,NaN,NaN,0.692786,...,NaN,0.650927,NaN,0.694099,NaN,0.619643,NaN,NaN,NaN,0.625644


## Algorithm

In [5]:
with open(paths.models_dir("trained_esn_ndvi.pickle"), "rb") as file:
  model = pickle.load(file)

In [14]:
signal: pd.Series = signal_df.iloc[0]
model: rpy.model.Model = model
num_features: int = 104
forecasted_steps: int = 52
step_size: int = 1

# TODO: Do we need to keep start and end dates for denoised signal?
denoised_signal_series = denoise_signal(
    signal, [
        resample_time_series,
        moving_std_filter,
        holt_winters_filter,
    ]
)

denoised_signal_dates = denoised_signal_series.index.to_numpy()
denoised_signal_dates = denoised_signal_dates.reshape(-1, 1)
X_dates = denoised_signal_dates[num_features:, :]

denoised_signal = denoised_signal_series.to_numpy()
X, _ = create_training_data(denoised_signal, num_features=num_features)


signal_forecaster = Forecaster(model, num_features=num_features)

i = 0
# i = X.shape[0] - forecasted_steps

warmup = X[i:i + num_features, :]


pred_signal = signal_forecaster.forecast(
    prediction_length=forecasted_steps,
    warmup_X=warmup,
    )
pred_dates = X_dates[i: i + forecasted_steps, :]

pred = pd.Series(pred_signal.flatten(), index=pred_dates.flatten())


In [15]:
pred

2016-07-03    0.783426
2016-07-10    0.777498
2016-07-17    0.764750
2016-07-24    0.768446
2016-07-31    0.765643
2016-08-07    0.771382
2016-08-14    0.773389
2016-08-21    0.768295
2016-08-28    0.772427
2016-09-04    0.760646
2016-09-11    0.750991
2016-09-18    0.729537
2016-09-25    0.725547
2016-10-02    0.730585
2016-10-09    0.730442
2016-10-16    0.744571
2016-10-23    0.749799
2016-10-30    0.762337
2016-11-06    0.751255
2016-11-13    0.755657
2016-11-20    0.756098
2016-11-27    0.759588
2016-12-04    0.756053
2016-12-11    0.739493
2016-12-18    0.727725
2016-12-25    0.715500
2017-01-01    0.726508
2017-01-08    0.722128
2017-01-15    0.724124
2017-01-22    0.729768
2017-01-29    0.725739
2017-02-05    0.723539
2017-02-12    0.717673
2017-02-19    0.730795
2017-02-26    0.715820
2017-03-05    0.719436
2017-03-12    0.714349
2017-03-19    0.702777
2017-03-26    0.697553
2017-04-02    0.687086
2017-04-09    0.688572
2017-04-16    0.687751
2017-04-23    0.714187
2017-04-30 

## Predictions data structure

In [29]:
from collections import namedtuple

In [30]:
Prediction = namedtuple("Prediction", "idx preds actual_signal")

In [31]:
sample_signal = signal_df.iloc[0]

In [32]:
sample_signal.name

(59, 1530)

In [33]:
example_pred = Prediction(idx=sample_signal.name,
                          preds=sample_signal,
                          actual_signal=sample_signal)

In [34]:
example_pred.actual_signal[pd.Timestamp("2016-01-30")]

0.8235955

In [ ]:
# Create pred store

## Wrapper class for predictions

Lazy loading of detection objects that allows pandas-like indexation

In [35]:
class PredictionsLazyWrapper:
  def __init__(self, pred_index, pred_store):
    self.__pred_index = pred_index
    self.__pred_store = pred_store
    self.__pred = None

  @property
  def pred(self):
    if self.__pred is None:
      self.__pred = self.__pred_store[self.__pred_index]

      return self.__pred